In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/cost2100'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cost2100/DATA_Hvalout.mat
/kaggle/input/cost2100/DATA_Htrainin.mat
/kaggle/input/cost2100/DATA_HtestFout_all.mat
/kaggle/input/cost2100/DATA_Htestout.mat
/kaggle/input/cost2100/DATA_Htestin.mat
/kaggle/input/cost2100/DATA_Hvalin.mat
/kaggle/input/cost2100/DATA_Htrainout.mat
/kaggle/input/cost2100/DATA_HtestFin_all.mat


In [54]:
from datetime import datetime
import sys
import traceback
from datetime import datetime
import sys
import traceback
import scipy.io as sio
import torch

In [55]:
class ConvBN(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size,groups=1):
        if not isinstance(kernel_size, int):
            padding = [(i - 1) // 2 for i in kernel_size]
        else:
            padding = (kernel_size - 1) // 2
        super(ConvBN, self).__init__(OrderedDict([
            ('conv', nn.Conv2d(in_planes, out_planes, kernel_size,
                               padding=padding)),
            ('bn', nn.BatchNorm2d(out_planes,eps=1e-03, momentum=0.99))
        ]))

        

class RefineNet(nn.Module):
    def __init__(self, img_channels=2):
        super(RefineNet, self).__init__()
        self.conv=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(img_channels,8,[1,7])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(8,16,[1,7])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,2,[1,7])),    
        ]))
        
        self.conv_1=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(img_channels,8,[1,5])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(8,16,[1,5])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,2, [1,5])),    
        ]))
        
        self.conv1x1 = ConvBN(4, 2, [1,7])
        self.Relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)
        
        
    def forward(self, x):
        ori_x = x.clone()

        # concatenate
        x_1 = self.conv(x)
        x_2 = self.conv_1(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.Relu(x)
        x =self.conv1x1(x)

        return self.Relu(x + ori_x)
    
    
    
    
class Encoder_Compression(nn.Module):
    def __init__(self):
        super(Encoder_Compression, self).__init__()
        self.conv=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(64,32,[1,7])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(32,16,[1,7])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,4,[1,7])),
        ]))
        
        self.conv_2=ConvBN(64,4,[1,7])
        self.conv_3=ConvBN(8,4,[1,7])
        
        
        self.LeakyRelu = nn.LeakyReLU(negative_slope=0.3, inplace=True)
               
    def forward(self, x):        
        # concatenate
        x_1= self.conv(x)
        x_2= self.conv_2(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.LeakyRelu(x)
        x = self.conv_3(x)
        #print("finish")
        ###### Send Feedback From User Equipement" 
        return self.LeakyRelu(x)
    
        
        
    
 
    
    

class CLLWCsiNet(nn.Module):
    def __init__(self,reduction=4,residual_num=2):
        super(CLLWCsiNet, self).__init__()
        total_size, in_channel, w, h = 2048, 2, 32, 32
        
        self.encoder_p1 = nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(2,2, [1,7])),
            ("LeakyReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x7",ConvBN(2,2,[7,1])),
            #("LeakyReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ]))
        
        self.encoder_p2= nn.Sequential(OrderedDict([
            ('conv1x3', ConvBN(2, 2, [1, 5])),
            ('LeakyRelu_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv3x1', ConvBN(2, 2, [5, 1])),
            ]))
        
        self.encoder_p3= nn.Sequential(OrderedDict([
            ('conv1x3', ConvBN(2, 2, [1, 3])),
            ('LeakyRelu_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv3x1', ConvBN(2, 2, [3, 1])),
            ]))
        
        self.con1x1=ConvBN(6, 2, [1,7])
        self.LeakyReLU=nn.LeakyReLU(negative_slope=0.3, inplace=True)
        
        ######################## CNN base Laten Space ########################################
        self.encoder_compression=Encoder_Compression()
        
        
        
        self.decoder_get_feedback_in_UE=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(4,8,[1,7])),
            ("LeakyRelu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv1x7",ConvBN(8,16,[1,7])),
            ("LeakyRelu_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x7",ConvBN(16,64,[1,7])),
             #("LeakyRelu_3",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ]))
        
        
        self.remove_AGN=nn.Sequential(OrderedDict([
            ("conv_1x7",ConvBN(4,8,[1,7])),
            ("LeakyReLu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv_7x1",ConvBN(8,16,[1,7])),
            ("LeakyReLu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv_3x3",ConvBN(16,64,[1,7])),
        ]))
        
        
        ###################################    refine Module #########################################
#         self.refine =  nn.Sequential(OrderedDict([
#             ("MultiResolutionRefineBlock", MultiResolutionRefineBlock()),
#             ("MultiResolutionRefineBlock", MultiResolutionRefineBlock())
#         ]))
        
        
        self.decoder_refine_net = nn.ModuleList([RefineNet(in_channel) for _ in range(residual_num)])
        
        self._last_cov=nn.Sequential(OrderedDict([
            ("firstcov2",ConvBN(2,2,[1,7])),
            ("activation",nn.Sigmoid())
        ]))
        
        
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d)):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        
    def adding_noise(self, x):
        # Ensure input is normalized to [0, 1]
        #assert x.min() >= 0 and x.max() <= 1, "Input must be in [0, 1] range. Normalize first!"
        
        # Compute signal power (assumes [0,1] range)
        signal_power = torch.mean(x**2)
        
        # Set SNR (e.g., 20 dB)
        snr_db = -10 
        snr_linear = 10 ** (snr_db / 10)
        
        # Calculate noise power
        noise_power = signal_power / snr_linear
        
        # Generate noise
        noise = torch.randn_like(x) * torch.sqrt(noise_power)
        
        # Add noise and clip to maintain [0,1] range
        noisy_x = torch.clamp(x + noise, 0, 1)
        
        return noisy_x
    
    
    
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        #print(x.size())
        x_1=self.encoder_p1(x)
        x_2=self.encoder_p2(x)
        x_3=self.encoder_p3(x)
        x=torch.cat((x_1,x_2,x_3),dim=1)
        x=self.con1x1(x)
        x=self.LeakyReLU(x)
        #print(x.size())
        x = x.contiguous().view(batch_size,64 ,1,32)
        #print(x.size())
        x=self.encoder_compression(x)
        x_noisy_feedback=self.adding_noise(x)
        y=self.remove_AGN(x_noisy_feedback)
        x=self.decoder_get_feedback_in_UE(x)
        x=x-y
        x=self.LeakyReLU(x)
        x = x.contiguous().view(batch_size,2 ,32,32)
        #x = self.refine(x)
        for layer in self.decoder_refine_net:
            x = layer(x)
        x=self._last_cov(x)
       
        
        return x

In [56]:
model = CLLWCsiNet()

In [57]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [58]:
pytorch_total_params

42608

In [69]:
import torch
#from Trans_Net.utils import *


state_dict = torch.load(r'/kaggle/input/cllwcsinet-noisy-channel/CLLWCsiNet-noisy-channel/cr16/outdoor/CLLWCsiNet_Net_model_500_outdoor_128_.pth',map_location=torch.device('cpu'))['model_state_dict']

/tmp/ipykernel_31/513177726.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'/kaggle/input/cllwcsinet-noisy-channel/CLLWCsiNet-noisy-channel/cr1

In [70]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [71]:
model.eval()

CLLWCsiNet(
  (encoder_p1): Sequential(
    (first_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (second_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
  )
  (encoder_p2): Sequential(
    (conv1x3): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyRelu_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv3x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)

In [72]:
model.to('cpu')

CLLWCsiNet(
  (encoder_p1): Sequential(
    (first_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (second_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
  )
  (encoder_p2): Sequential(
    (conv1x3): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyRelu_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv3x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)

In [73]:
criterion = nn.MSELoss().to('cpu')

In [ ]:
model._fc_binarization()

In [74]:
envir = 'outdoor'  # 'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2
img_total = img_height * img_width * img_channels
# network params
#residual_num = 2
encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [75]:
# envir = 'indoor'  # 'indoor' or 'outdoor'
# # image params
# img_height = 32
# img_width = 32
# img_channels = 2
# img_total = img_height * img_width * img_channels
# # network params
# residual_num = 2
# encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32
# Data loading
#/kaggle/input/cost2100/DATA_HtestFout_all.mat
if envir == 'indoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainin.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalin.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestin.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFin_all.mat')
    X_test = mat['HF_all']  # array

elif envir == 'outdoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainout.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalout.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestout.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFout_all.mat')
    X_test = mat['HF_all']  # array

In [76]:
import numpy as np

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train_length = len(x_train)
x_val_length = len(x_val)
x_test_length = len(x_test)

x_train = np.reshape(x_train, (x_train_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (x_val_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (x_test_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

x_train = torch.tensor(x_train)
x_val = torch.tensor(x_val)
x_test = torch.tensor(x_test)

In [77]:
import math
with torch.no_grad():

    #torch.cuda.empty_cache()
    model.eval()
    device='cpu'
    x_hat = model(x_test)
    #torch.quantization.convert(quantized_model, inplace=True)
    x_test = x_test.to('cpu')
    x_hat=x_hat.to('cpu')

    # Calcaulating the NMSE and rho
    # if envir == 'indoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFin_all.mat')
    #     X_test = mat['HF_all']  # array

    # elif envir == 'outdoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFout_all.mat')
    #     X_test = mat['HF_all']  # array

    #X_test = torch.tensor(X_test)
    #X_test = torch.reshape(X_test, (len(X_test), img_height, 125))
    x_test_real = torch.reshape(x_test[:, 0, :, :], (len(x_test), -1))
    x_test_imag = torch.reshape(x_test[:, 1, :, :], (len(x_test), -1))
    x_test_C = x_test_real - 0.5 + 1j * (x_test_imag - 0.5)
    x_hat_real = torch.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
    x_hat_imag = torch.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
    x_hat_C = x_hat_real - 0.5 + 1j * (x_hat_imag - 0.5)
    x_hat_F = torch.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
    X_hat = torch.fft.fft(torch.cat((x_hat_F, torch.zeros((len(x_hat_C), img_height, 257 - img_width))), axis=2), axis=2)
    X_hat = X_hat[:, :, 0:125]

    #n1 = torch.sqrt(torch.sum(torch.conj(X_test) * X_test, axis=1))
    #n2 = torch.sqrt(torch.sum(torch.conj(X_hat) * X_hat, axis=1))
    #aa = abs(torch.sum(torch.conj(X_test) * X_hat, axis=1))
    #rho = torch.mean(aa / (n1 * n2), axis=1)
    X_hat = torch.reshape(X_hat, (len(X_hat), -1))
    #X_test = torch.reshape(X_test, (len(X_test), -1))
    power = torch.sum(abs(x_test_C) ** 2, axis=1)
    power_d = torch.sum(abs(X_hat) ** 2, axis=1)
    mse = torch.sum(abs(x_test_C - x_hat_C) ** 2, axis=1)
    NMSE = 10 * math.log10(torch.mean(mse / power))
    #Correlation = torch.mean(rho).item().real

    # print("In " + envir + " environment")
    # print("When dimension is", encoded_dim)
    print("NMSE is ", NMSE)
    #print("Correlation is ", Correlation)
#
# file = 'CsiNet_' + (envir) + '_dim' + str(encoded_dim) + time.strftime('_%m_%d_%H_%M')
# outfile = "result/result_%s.mat" % file
# savemat(outfile, {'train_loss_history': train_loss_history,
#                   'val_loss_history': val_loss_history,
#                   'training_time': training_time,
#                   'NMSE': NMSE,
#                   'Correlation': Correlation})

NMSE is  -2.088769005431192


In [78]:
# ====== ADD THIS NEW CELL FOR TIERED CPU/GPU BENCHMARK ======
import time
import torch
from tabulate import tabulate

def run_inference_benchmark(model, input_shape=(2, 32, 32), warmup=10, repetitions=100):
    """Run comprehensive inference benchmarks on CPU and GPU"""
    results = []
    model.eval()
    
    # CPU Benchmark
    device = 'cpu'
    model.to(device)
    dummy_input = torch.randn(1, *input_shape).to(device)
    
    # Warm-up
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(dummy_input)
    
    # Measurement
    start_time = time.time()
    with torch.no_grad():
        for _ in range(repetitions):
            _ = model(dummy_input)
    cpu_time = (time.time() - start_time) * 1000 / repetitions
    
    results.append(('CPU', 'Single', cpu_time, 1000/cpu_time))
    
    # GPU Benchmark (if available)
    if torch.cuda.is_available():
        device = 'cuda'
        model.to(device)
        dummy_input = dummy_input.to(device)
        
        # Warm-up with synchronization
        with torch.no_grad():
            for _ in range(warmup):
                _ = model(dummy_input)
        torch.cuda.synchronize()
        
        # Measurement
        start_time = time.time()
        with torch.no_grad():
            for _ in range(repetitions):
                _ = model(dummy_input)
        torch.cuda.synchronize()
        gpu_time = (time.time() - start_time) * 1000 / repetitions
        
        results.append(('GPU', 'Single', gpu_time, 1000/gpu_time))
        results.append(('Speedup', '', f"{cpu_time/gpu_time:.2f}x", ''))
    
    return results

# Run benchmarks
print("\n=== Stage 1: Device-Specific Single Sample Benchmark ===")
benchmark_results = run_inference_benchmark(model)

# Print formatted results
headers = ["Device", "Test", "Time (ms)", "Samples/sec"]
print(tabulate(benchmark_results, headers=headers, floatfmt=".2f"))

# Batch Processing Benchmark
def batch_benchmark(model, batch_sizes=[1, 8, 16, 32, 64]):
    batch_results = []
    for device in ['cpu', 'cuda'] if torch.cuda.is_available() else ['cpu']:
        if device == 'cuda' and not torch.cuda.is_available():
            continue
            
        model.to(device)
        for bs in batch_sizes:
            dummy_batch = torch.randn(bs, 2, 32, 32).to(device)
            
            # Warm-up
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            # Measure
            start = time.time()
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            total_time = (time.time() - start) * 1000
            time_per_sample = total_time / bs
            
            batch_results.append([
                device.upper(),
                bs,
                f"{total_time:.2f}",
                f"{time_per_sample:.2f}",
                f"{1000/time_per_sample:.2f}"
            ])
    return batch_results

print("\n=== Stage 2: Batch Processing Benchmark ===")
batch_results = batch_benchmark(model)
batch_headers = ["Device", "Batch Size", "Total (ms)", "Per Sample (ms)", "Samples/sec"]
print(tabulate(batch_results, headers=batch_headers))


=== Stage 1: Device-Specific Single Sample Benchmark ===
Device    Test    Time (ms)           Samples/sec
--------  ------  ------------------  ------------------
CPU       Single  3.2305550575256348  309.54432974930495
GPU       Single  4.014348983764648   249.10639409885135
Speedup           0.80x

=== Stage 2: Batch Processing Benchmark ===
Device      Batch Size    Total (ms)    Per Sample (ms)    Samples/sec
--------  ------------  ------------  -----------------  -------------
CPU                  1          3.1                3.1          322.32
CPU                  8          8.41               1.05         951.44
CPU                 16         11.83               0.74        1352.76
CPU                 32         16.05               0.5         1993.96
CPU                 64         29.99               0.47        2134.2
CUDA                 1          4.08               4.08         245.05
CUDA                 8          3.97               0.5         2014.56
CUDA          